In [ ]:
import os
import sys
import glob
import pandas as pd
import logging
import json # 상세 로그(dict) 출력을 위해
from typing import Dict, Any, List

# --- 프로젝트 루트 경로 설정 ---
try:
    PROJECT_ROOT = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
except NameError:
    PROJECT_ROOT = os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd())))

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
    print(f"Added project root to path: {PROJECT_ROOT}")

# --- src 모듈 임포트 ---
try:
    from src.utils import load_config, load_jsonl
    # 평가 함수 임포트 (사례 탐색용)
    from src.evaluation import calculate_qa_metrics_for_item # PreciseWikiQA용
    # from src.evaluation import evaluate_factscore_official # FactScore용 (구현 필요)
except ImportError as e:
    print(f"Error importing src modules: {e}")

# 로깅 및 스타일 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(module)s - %(message)s')
logger = logging.getLogger(__name__)
pd.set_option('display.max_colwidth', 500) # DataFrame에서 긴 텍스트 잘리지 않게

In [ ]:
# --- 1. 모든 결과 파일 로드 ---
CONFIG_PATH = os.path.join(PROJECT_ROOT, "configs", "config.yaml")
config = load_config(CONFIG_PATH)
RESULTS_BASE_DIR = os.path.join(PROJECT_ROOT, config.get('results_base_dir', 'results'))
print(f"Loading all results from: {RESULTS_BASE_DIR}")

all_results_long = []
# `results/` 하위의 모든 .jsonl 파일 탐색 (hyperparam_tuning 제외)
file_paths = glob.glob(os.path.join(RESULTS_BASE_DIR, "**", "*.jsonl"), recursive=True)
file_paths = [p for p in file_paths if 'hyperparam_tuning' not in p.replace("\\", "/")]

if not file_paths:
    logger.error(f"[오류] {RESULTS_BASE_DIR}에서 분석할 결과 파일을 찾을 수 없습니다.")
else:
    print(f"Found {len(file_paths)} result files to analyze.")

# --- 2. 파일명 파싱 및 데이터 통합 ---
# (3_main_results_analysis의 로직과 유사)
import re
serc_regex = re.compile(r"serc_t(\d+)_mf(\d+)")
dense_regex = re.compile(r"serc_dense_iterative_t(\d+)") # dense-iterative
dense_1pass_regex = re.compile(r"serc_dense_1pass") # dense-1pass

for f_path in file_paths:
    try:
        filename = os.path.basename(f_path)
        parts = f_path.replace("\\", "/").split('/')
        dataset_name = parts[-2]
        model_name = parts[-3].replace('_', '/')
        
        method = "unknown"
        t_max = None
        max_facts = None
        
        if filename.startswith("baseline"):
            method = "Baseline"
        elif filename.startswith("cove"):
            method = "CoVe"
        elif dense_1pass_regex.search(filename):
             method = "SERC (Dense, 1-pass)"
             t_max = 1
        elif dense_regex.search(filename):
            method = "SERC (Dense, Iter)"
            match_dense = dense_regex.search(filename)
            if match_dense: t_max = int(match_dense.group(1))
        elif filename.startswith("serc"):
            match_serc = serc_regex.search(filename)
            if match_serc:
                t_max = int(match_serc.group(1))
                max_facts = int(match_serc.group(2))
                method = "SERC (1-pass)" if t_max == 1 else "SERC (Iterative)"
            else:
                method = "SERC (Unknown)"
        else:
             continue # 모르는 파일

        # 결과 파일 로드
        results_data = load_jsonl(f_path)
        
        for item in results_data:
            # 평가에 필요한 키 추출
            query = item.get('query', item.get('question'))
            # data_loader가 저장한 정답 (e.g., ['John F. Kennedy'] or ['No...'])
            ground_truth_list = item.get('answers', item.get('correct_answers_truthfulqa'))
            # 모델의 최종 답변
            final_output = item.get("method_result", {}).get("final_output")
            # SERC의 상세 로그 (분석용)
            serc_history_log = item.get("method_result", {}).get("serc_result", {})
            
            all_results_long.append({
                "model": model_name,
                "dataset": dataset_name,
                "method": method,
                "t_max": t_max,
                "max_facts": max_facts,
                "query": query,
                "ground_truth": ground_truth_list, # 정답 리스트
                "final_output": final_output, # 모델 예측
                "serc_history_log": serc_history_log # SERC 상세 로그
            })

    except Exception as e:
        logger.error(f"Error processing file {f_path}: {e}", exc_info=True)

# --- 3. 최종 통합 데이터프레임 생성 ---
df_all = pd.DataFrame(all_results_long)

if df_all.empty:
    logger.error("처리할 결과 데이터가 없습니다. 스크립트를 종료합니다.")
else:
    print("\n--- 모든 결과 데이터 통합 완료 ---")
    print(f"Total entries loaded: {len(df_all)}")
    print("DataFrame Info:")
    df_all.info()
    print("\nAvailable Methods:")
    print(df_all['method'].value_counts())
    print("\nAvailable Datasets:")
    print(df_all['dataset'].value_counts())
    display(df_all.head())

In [ ]:
# --- 사례 탐색(Case Finding)을 위한 평가 함수 ---

def evaluate_single_item(prediction, ground_truths, dataset_name):
    """단일 항목의 정답 여부를 True/False로 반환합니다 (간단 버전)"""
    if prediction is None or ground_truths is None:
        return False
        
    if 'precisewikiqa' in dataset_name.lower():
        # EM/F1 기반 평가
        metrics = calculate_qa_metrics_for_item(prediction, ground_truths)
        return metrics['em'] > 0.99 # EM 1.0
    
    elif 'truthfulqa' in dataset_name.lower():
        # TruthfulQA 평가는 복잡함.
        # 여기서는 임시로 'correct_answers'에 예측이 포함되는지 확인
        # TODO: 공식 TruthfulQA 평가 로직 연동 필요
        norm_pred = prediction.strip().lower()
        norm_correct = str(ground_truths).lower() # ground_truths가 문자열일 수 있음
        return norm_pred in norm_correct and norm_pred not in str(item.get('incorrect_answers_truthfulqa', '')).lower()
        
    elif 'longwiki' in dataset_name.lower():
        # FactScore는 항목별 True/False 반환이 어려움
        # 정성 분석에서는 수동 비교가 필요할 수 있음
        # 여기서는 임시로 '오류 없음'을 가정 (수동 분석 필요)
        return pd.NA # N/A (Not Applicable)
    
    return False

if not df_all.empty:
    # --- 4. 분석/비교를 위해 데이터프레임 피봇(Pivot) ---
    # (쿼리/모델/데이터셋별로 각 방법론의 결과를 열로 펼침)
    
    # 평가 수행 (True/False)
    # (LongWiki는 FactScore 필요하므로 여기서는 QA 벤치마크만 예시로 수행)
    qa_datasets = [d for d in df_all['dataset'].unique() if 'precisewikiqa' in d.lower() or 'truthfulqa' in d.lower()]
    if qa_datasets:
        df_qa = df_all[df_all['dataset'].isin(qa_datasets)].copy()
        df_qa['is_correct'] = df_qa.apply(
            lambda row: evaluate_single_item(row['final_output'], row['ground_truth'], row['dataset']),
            axis=1
        )
    else:
        df_qa = pd.DataFrame(columns=['query', 'model', 'dataset', 'method', 'is_correct'])
        logger.warning("QA 데이터셋(PreciseWikiQA, TruthfulQA) 결과가 없어 사례 탐색이 제한됩니다.")


    # LongWiki 데이터 (FactScore는 3_main_results... 에서 계산된 점수 사용 가정)
    df_longwiki = df_all[df_all['dataset'] == 'hallulens_longwiki'].copy()

    print("\n--- QA 데이터(PreciseWikiQA, TruthfulQA) 평가 완료 (간단 버전) ---")
    display(df_qa[['model', 'dataset', 'method', 'is_correct']].head())

In [ ]:
# --- 5. 상세 로그 출력을 위한 헬퍼 함수 ---

def display_serc_log(serc_history_log: Dict[str, Any]):
    """SERC 실행 기록(history dict)을 단계별로 예쁘게 출력합니다."""
    
    if not serc_history_log:
        print("[[ SERC 로그가 없거나 비어있습니다. ]]")
        return
        
    print(f"--- [Query] ---\n{serc_history_log.get('query')}\n")
    print(f"--- [Initial Baseline (t=0)] ---\n{serc_history_log.get('initial_baseline')}\n")
    
    cycles = serc_history_log.get('cycles', [])
    if not cycles:
        print("[[ 실행된 사이클이 없습니다. (e.g., 사실 추출 실패) ]]")
        
    for cycle in cycles:
        t = cycle.get('cycle')
        print(f"\n{'='*20} [ Cycle {t} ] {'='*20}")
        
        # 2. 사실 추출
        facts = cycle.get('steps', {}).get('2_fact_extraction', {}).get('parsed_facts', {})
        print(f"\n[2. Fact Extraction] (총 {len(facts)}개)")
        for fi, f_text in facts.items():
            print(f"  - {fi}: {f_text}")
            
        # 3. 신드롬 생성
        syndrome_step = cycle.get('steps', {}).get('3_syndrome_generation', {})
        if not syndrome_step:
             # (Dense 모드일 경우)
             syndrome_step = cycle.get('steps', {}).get('3_syndrome_generation_dense', {})
             print("\n[3. Syndrome Generation (Dense Mode)]")
        else:
             print("\n[3. Syndrome Generation (Low-Density Mode)]")
        
        # 3a. 태깅
        tags = syndrome_step.get('3a_tags', {})
        if tags:
            print("  [3a. Tags]:")
            print("    " + ", ".join([f"{fi}:'{tag}'" for fi, tag in tags.items()]))
            
        # 3b. 그룹화
        groups = syndrome_step.get('3b2_chunked_groups', {})
        if groups:
            print("  [3b. Groups]:")
            for tag, f_ids in groups.items():
                print(f"    - {tag}: {f_ids}")
                
        # 3c/d. 검증 및 신드롬
        validations = syndrome_step.get('3c_3d_details', [])
        syndrome_found = {}
        if validations:
            print("  [3c/d. Validation & Syndrome]:")
            for detail in validations:
                # (Dense 모드 로그)
                if 'question' in detail and 'verified_answer' in detail:
                    print(f"    - Fact: {detail.get('fact_id')}")
                    print(f"      Q: {detail.get('question')}")
                    print(f"      A: {detail.get('verified_answer')}")
                    print(f"      Result: {detail.get('result')}")
                    if detail.get('result') == "[예]":
                        syndrome_found[detail.get('fact_id')] = detail.get('fact_text')
                # (Low-Density 모드 로그 - 그룹별)
                elif 'tag' in detail:
                    print(f"    - Group: '{detail.get('tag')}'")
                    print(f"      Q: {detail.get('question')}")
                    print(f"      A: {detail.get('verified_answer')}")
                    for val_item in detail.get('validations', []):
                        print(f"        - {val_item.get('fact_id')}: {val_item.get('fact_text')[:50]}... -> {val_item.get('result')}")
                        if val_item.get('result') == "[예]":
                            syndrome_found[val_item.get('fact_id')] = val_item.get('fact_text')
        
        if not syndrome_found:
             print("\n  [4c. Convergence Check]: 신드롬 없음 (No Syndrome).")
        
        # 5. 교정
        corrections = cycle.get('steps', {}).get('5_correction', [])
        if corrections:
            print("\n[5. Correction]:")
            for corr_item in corrections:
                print(f"  - Targeting Fact: {corr_item.get('fact_id')} ('{corr_item.get('original_fact', '')[:50]}...')")
                print(f"    Status: {corr_item.get('status')}")
                print(f"    Found Sentence: {corr_item.get('found_sentence')}")
                print(f"    Corrected Fact: {corr_item.get('corrected_fact')}")
                print(f"    Rewritten Sentence: {corr_item.get('rewritten_sentence')}")
        
        print(f"\n--- [Baseline After Cycle {t}] ---\n{cycle.get('baseline_after_cycle')}\n")

    print(f"\n{'='*20} [ Final Result ] {'='*20}")
    print(f"--- [Final Baseline] ---\n{serc_history_log.get('final_baseline')}")
    print(f"Termination: {serc_history_log.get('termination_reason')}")


# --- 6. 사례 탐색기 (Case Finder) ---
def find_cases(df_analysis, model, dataset, case_type):
    """연구 계획서 4.6.2 기준에 따라 사례 탐색"""
    
    # 모델/데이터셋 필터링
    df_filtered = df_analysis[
        (df_analysis['model'] == model) &
        (df_analysis['dataset'] == dataset)
    ].copy()
    
    if df_filtered.empty:
        print(f"'{model}' / '{dataset}'에 대한 결과를 찾을 수 없습니다.")
        return pd.DataFrame()

    # (쿼리별로 방법론 피봇)
    df_pivot = df_filtered.pivot_table(
        index='query', # 쿼리 기준
        columns='method', # 방법론을 열로
        values=['is_correct', 'final_output', 'serc_history_log'], # 필요한 값
        aggfunc='first' # 중복 제거 (보통 쿼리당 하나)
    ).reset_index()
    
    # 컬럼 이름 재구성 (e.g., ('is_correct', 'Baseline'))
    df_pivot.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else col for col in df_pivot.columns.values]
    
    print(f"--- {model} / {dataset} / {case_type} 사례 탐색 ---")

    try:
        if case_type == "RQ3_ITERATIVE_SUCCESS" and 'is_correct_SERC (1-pass)' in df_pivot.columns:
            # (Baseline=False) AND (1-pass=False) AND (Iterative=True)
            cases = df_pivot[
                (df_pivot['is_correct_Baseline'] == False) &
                (df_pivot['is_correct_SERC (1-pass)'] == False) &
                (df_pivot['is_correct_SERC (Iterative)'] == True)
            ]
            
        elif case_type == "FAILURE_ANALYSIS" and 'is_correct_SERC (Iterative)' in df_pivot.columns:
            # (Iterative=False)
            cases = df_pivot[
                (df_pivot['is_correct_SERC (Iterative)'] == False)
            ]
            
        elif case_type == "ERROR_AMPLIFICATION" and 'is_correct_Baseline' in df_pivot.columns:
            # (Baseline=True) AND (Iterative=False)
             cases = df_pivot[
                (df_pivot['is_correct_Baseline'] == True) &
                (df_pivot['is_correct_SERC (Iterative)'] == False)
            ]
        else:
            print(f"'{case_type}' 사례를 찾을 수 없거나, 필요한 방법론(e.g., SERC (1-pass))의 결과가 없습니다.")
            return pd.DataFrame()
            
        print(f"Found {len(cases)} cases.")
        return cases
        
    except KeyError as e:
        print(f"분석에 필요한 컬럼이 부족합니다: {e}")
        return pd.DataFrame()

In [ ]:
# --- 7. 분석 실행 ---
# (분석하려는 모델과 데이터셋 지정)
TARGET_MODEL = "meta-llama/Llama-3.1-8B-Instruct" # 예시 모델
TARGET_DATASET = "hallulens_precisewikiqa" # 예시 QA 데이터셋

# (df_qa는 셀 3에서 QA 데이터셋만 필터링하여 'is_correct'를 계산한 DataFrame임)
if 'df_qa' in locals() and not df_qa.empty:
    
    # --- 사례 1: RQ3 (반복 교정) 증명 사례 ---
    # (이 사례는 T_max=1 (1-pass)와 T_max=2+ (Iterative) 결과를 모두 실행해야 찾을 수 있음)
    rq3_cases = find_cases(df_qa, TARGET_MODEL, TARGET_DATASET, "RQ3_ITERATIVE_SUCCESS")
    display(rq3_cases.head())

    # --- 사례 2: 교정 실패 사례 ---
    failure_cases = find_cases(df_qa, TARGET_MODEL, TARGET_DATASET, "FAILURE_ANALYSIS")
    if not failure_cases.empty:
        print("\n--- 교정 실패 사례 (상세 로그 1개) ---")
        
        # 첫 번째 실패 사례의 상세 로그 가져오기
        # (df_all에서 원본 로그를 찾아야 함)
        failed_query = failure_cases.iloc[0]['query']
        
        failed_log_entry = df_all[
            (df_all['query'] == failed_query) &
            (df_all['model'] == TARGET_MODEL) &
            (df_all['method'] == 'SERC (Iterative)') # 실패한 SERC Iterative 로그
        ]
        
        if not failed_log_entry.empty:
            failed_log_dict = failed_log_entry.iloc[0]['serc_history_log']
            display_serc_log(failed_log_dict) # 상세 로그 출력
        else:
            print("실패 사례에 대한 상세 로그를 찾을 수 없습니다.")
            
    # --- 사례 3: 오류 증폭 사례 ---
    amp_cases = find_cases(df_qa, TARGET_MODEL, TARGET_DATASET, "ERROR_AMPLIFICATION")
    display(amp_cases.head())
    
    # (LongWiki (df_longwiki)에 대한 분석은 FactScore 점수 기반으로 유사하게 수행)
    
else:
    print("\n분석할 QA 데이터가 없습니다. 셀 2와 3을 먼저 실행하세요.")